## ENVIRONMENT VARIABLES

In [1]:
EN_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/"

In [2]:
QUY_ES_CORPUS_DIR = "/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish"

In [3]:
tokenizer_path = "/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/"

In [4]:
%env TOKENIZER_PATH = /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en

env: TOKENIZER_PATH=/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en


In [5]:
mkdir /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en

mkdir: cannot create directory '/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en': File exists


In [6]:
tokenized_path_es_en = "/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/"
tokenized_path_quy_es = "/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/quy-es/"

In [7]:
%env TOKENIZED_PATH_ES_EN = /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en
%env TOKENIZED_PATH_QUY_ES = /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/quy-es

env: TOKENIZED_PATH_ES_EN=/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en
env: TOKENIZED_PATH_QUY_ES=/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/quy-es


In [8]:
%env BIN_DIR = /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin

env: BIN_DIR=/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin


In [9]:
%env MODEL_DIR = /storage/master-thesis/models/quy-es+es-en

env: MODEL_DIR=/storage/master-thesis/models/quy-es+es-en


In [14]:
! locale

locale: Cannot set LC_CTYPE to default locale: No such file or directory
locale: Cannot set LC_MESSAGES to default locale: No such file or directory
locale: Cannot set LC_ALL to default locale: No such file or directory
LANG=en_US.UTF-8
LANGUAGE=
LC_CTYPE="en_US.UTF-8"
LC_NUMERIC="en_US.UTF-8"
LC_TIME="en_US.UTF-8"
LC_COLLATE="en_US.UTF-8"
LC_MONETARY="en_US.UTF-8"
LC_MESSAGES="en_US.UTF-8"
LC_PAPER="en_US.UTF-8"
LC_NAME="en_US.UTF-8"
LC_ADDRESS="en_US.UTF-8"
LC_TELEPHONE="en_US.UTF-8"
LC_MEASUREMENT="en_US.UTF-8"
LC_IDENTIFICATION="en_US.UTF-8"
LC_ALL=en_US.UTF-8


In [13]:
! update-locale LANG=en_US.UTF-8 LANGUAGE=en.UTF-8

%env LANG=en_US.UTF-8
%env LC_CTYPE=en_US.UTF-8
%env LC_ALL=en_US.UTF-8

/bin/sh: 1: update-locale: not found
env: LANG=en_US.UTF-8
env: LC_CTYPE=en_US.UTF-8
env: LC_ALL=en_US.UTF-8


In [10]:
! cat /etc/rc.conf

cat: /etc/rc.conf: No such file or directory


## Libraries

In [9]:
pip install tokenizers==0.10.3

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install fairseq

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Converting tatoeba format to standard format

In [ ]:
import csv

with open(EN_ES_CORPUS_DIR + "original/valid.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "dev.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "dev.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
    en.close()
    es.close()
    
with open(EN_ES_CORPUS_DIR + "original/dev.txt", encoding='utf8') as f:
    es = open(EN_ES_CORPUS_DIR + "train.es", 'w', encoding='utf8')
    en = open(EN_ES_CORPUS_DIR + "train.en", 'w', encoding='utf8')
    for line in csv.reader(f, delimiter="\t"):
        if "\n" in line[2] or "\n" in line[3]:
            continue
        if len(line) != 4:
            continue
            
        es.write(line[3] + "\n")
        en.write(line[2] + "\n")
        
    en.close()
    es.close()
        

### Tokenizing data

In [10]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

en_es_quy_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [11]:
from tokenizers.trainers import WordPieceTrainer

en_es_quy_trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=30000, show_progress=True)

In [12]:
from tokenizers.pre_tokenizers import Whitespace

en_es_quy_tokenizer.pre_tokenizer = Whitespace()

en_files = [f"/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/{split}.en" for split in ["dev", "train"]]
es_files = [f"/notebooks/master-thesis/corpora/tatoeba-challenge/en-es/{split}.es" for split in ["dev", "train"]]
quy_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/{split}.quy" for split in ["dev", "dict", "train"]]
es2_files = [f"/notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/{split}.es" for split in ["dev", "dict", "train"]]

files = en_files + es_files + es2_files + quy_files
en_es_quy_tokenizer.train(files= files, trainer=en_es_quy_trainer)

In [13]:
from pathlib import Path



def tokenize_files(tokenizer, files, extension, output_path):
    for file in files:
        print(f"Reading file {file}")
        with open(file, encoding='utf8') as f:
          lines = f.readlines()
          tokenized_lines = tokenizer.encode_batch(lines)
          tokenized_name = Path(file).stem
          tokenized_name = output_path + tokenized_name + "." + extension
          print(tokenized_name)
          with open(tokenized_name, 'w', encoding='utf8') as wf:

            wf.writelines([" ".join(t.tokens) + "\n" for t in tokenized_lines])

In [21]:
en_es_quy_tokenizer.save(tokenizer_path + "quy-es-en-tokenizer.json")

In [22]:
print(en_es_quy_tokenizer)

In [23]:
tok = en_es_quy_tokenizer.encode("we hold this truth to be self evident that everyone is created equal?")
print(tok.tokens)


['we', 'hold', 'this', 'truth', 'to', 'be', 'self', 'evid', '##ent', 'that', 'everyone', 'is', 'created', 'equal', '?']


In [28]:
tokenize_files(en_es_quy_tokenizer, en_files, "en", tokenized_path_es_en)
tokenize_files(en_es_quy_tokenizer, es_files, "es", tokenized_path_es_en)

tokenize_files(en_es_quy_tokenizer, quy_files, "quy", tokenized_path_quy_es)
tokenize_files(en_es_quy_tokenizer, es2_files, "es", tokenized_path_quy_es)

Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.en
/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/dev.en
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/train.en
/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/train.en
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/dev.es
/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/dev.es
Reading file /notebooks/master-thesis/corpora/tatoeba-challenge/en-es/train.es
/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/train.es
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/dev.quy
/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/quy-es/dev.quy
Reading file /notebooks/master-thesis/corpora/americasnlp2021/data/quechua-spanish/dict.quy
/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/quy-es/dict.quy
Reading file /notebooks/master-the

In [ ]:
! cat /storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/es-en/dev.en


## Binarizing data

In [29]:
### Removing previous dict files
! rm $BIN_DIR/dict.quy.txt
! rm $BIN_DIR/dict.es.txt
! rm $BIN_DIR/dict.en.txt

rm: cannot remove '/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin/dict.quy.txt': No such file or directory
rm: cannot remove '/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin/dict.es.txt': No such file or directory
rm: cannot remove '/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin/dict.en.txt': No such file or directory


In [30]:
## Concatenating all training data
! cat $TOKENIZED_PATH_QUY_ES/train.quy $TOKENIZED_PATH_QUY_ES/train.es $TOKENIZED_PATH_ES_EN/train.es $TOKENIZED_PATH_ES_EN/train.en > $BIN_DIR/train.all

In [31]:
ls

tokenizers/


In [32]:
! fairseq-preprocess --source-lang all \
    --trainpref $BIN_DIR/train \
    --destdir $BIN_DIR \
    --workers 20 \
    --only-source

2021-10-21 18:50:24 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='all', srcdict=None, target_lang=None, task='translation', tensorboard_logdir=None, testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresh

In [33]:
! fairseq-preprocess --source-lang quy --target-lang es \
    --trainpref $TOKENIZED_PATH_QUY_ES/train --validpref $TOKENIZED_PATH_QUY_ES/dev \
    --destdir $BIN_DIR \
    --srcdict $BIN_DIR/dict.all.txt \
    --tgtdict $BIN_DIR/dict.all.txt \
    --workers 20

2021-10-21 18:50:42 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='quy', srcdict='/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin/dict.all.txt', target_lang='es', task='translation', tensorboard_logdir=None,

### Reusing es dict from quy-es preprocessing

In [34]:
! fairseq-preprocess --source-lang es --target-lang en \
    --trainpref $TOKENIZED_PATH_ES_EN/train --validpref $TOKENIZED_PATH_ES_EN/dev \
    --destdir $BIN_DIR \
    --srcdict $BIN_DIR/dict.all.txt \
    --tgtdict $BIN_DIR/dict.all.txt \
    --workers 20

2021-10-21 18:50:51 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='es', srcdict='/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin/dict.all.txt', target_lang='en', task='translation', tensorboard_logdir=None, 

## Training model

In [10]:
! fairseq-train $BIN_DIR \
    --arch=transformer --share-all-embeddings \
    --task translation_multi_simple_epoch --lang-pairs quy-es,es-en \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --restore-file $MODEL_DIR/checkpoint_last.pt \
    --save-dir $MODEL_DIR/ \
    --keep-last-epochs 2 \
    --reset-optimizer \
    --encoder-langtok "src" \
    --decoder-langtok \
    --fp16


2021-10-23 23:04:19 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/storage/master-thesis/models/quy-es+es-en/tokenizers/quy-es-en/bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_langtok=True, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=Fals

In [34]:
! echo $MODEL_DIR

/storage/master-thesis/models/quy-es+es-en


In [36]:
cd /storage/master-thesis/models/quy-es+es-en

/storage/master-thesis/models/quy-es+es-en


In [38]:
! rm checkpoint108.pt
! rm checkpoint109.pt
! rm checkpoint_best.pt
! rm checkpoint_last.pt

In [29]:
! rm dict.en.txt
! rm dict.es.txt
! rm dict.quy.txt

In [84]:
! du -shc /notebooks/*

2.5K	/notebooks/CITATION.cff
5.5K	/notebooks/CODE_OF_CONDUCT.md
15K	/notebooks/CONTRIBUTING.md
19K	/notebooks/ISSUES.md
12K	/notebooks/LICENSE
512	/notebooks/MANIFEST.in
3.5K	/notebooks/Makefile
41K	/notebooks/README.md
41K	/notebooks/README_zh-hans.md
42K	/notebooks/README_zh-hant.md
12K	/notebooks/docker
4.6M	/notebooks/docs
5.0M	/notebooks/examples
8.5K	/notebooks/hubconf.py
7.8G	/notebooks/master-thesis
1.5K	/notebooks/model_cards
9.5K	/notebooks/notebooks
512	/notebooks/pyproject.toml
64K	/notebooks/scripts
1.0K	/notebooks/setup.cfg
13K	/notebooks/setup.py
13M	/notebooks/src
731K	/notebooks/templates
4.5K	/notebooks/test quy-es -> es-en model.ipynb
6.8M	/notebooks/tests
147K	/notebooks/train es-en model.ipynb
158K	/notebooks/train quy-es + es-en model.ipynb
160K	/notebooks/utils
3.5K	/notebooks/valohai.yaml
7.9G	total


In [85]:
! du -shc /notebooks/master-thesis/*

512	/notebooks/master-thesis/Untitled.ipynb
362M	/notebooks/master-thesis/corpora
7.5G	/notebooks/master-thesis/models
7.8G	total


In [86]:
! du -shc /notebooks/master-thesis/models/*

1.6G	/notebooks/master-thesis/models/es-en
802M	/notebooks/master-thesis/models/quy-es
5.2G	/notebooks/master-thesis/models/quy-es+es-en
7.5G	total


In [78]:
! du -shc /*

19M	/apex
5.0M	/bin
4.0K	/boot
24K	/content
^C


In [89]:
ls

 CITATION.cff         docker/          setup.py
 CODE_OF_CONDUCT.md   docs/            src/
 CONTRIBUTING.md      examples/        templates/
 ISSUES.md            hubconf.py      'test quy-es -> es-en model.ipynb'
 LICENSE              master-thesis/   tests/
 MANIFEST.in          model_cards/    'train es-en model.ipynb'
 Makefile             notebooks/      'train quy-es + es-en model.ipynb'
 README.md            pyproject.toml   utils/
 README_zh-hans.md    scripts/         valohai.yaml
 README_zh-hant.md    setup.cfg


/storage/master-thesis
